<a href="https://colab.research.google.com/github/Heisnotanimposter/ObjectDetection_with_Server/blob/main/PersonSpeedvision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Sun Aug 18 13:10:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install -q supervision ultralytics
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.0/869.0 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.0 MB/s eta 0:00:00


In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
import supervision as sv
from collections import defaultdict, deque
from google.colab.patches import cv2_imshow

# Load the YOLO model
model = YOLO("yolov8s.pt")  # Use the appropriate YOLOv8 model variant

# Set up video paths
SOURCE_VIDEO_PATH = "/content/drive/MyDrive/Team7dataset/Team7Shared/140sNightShinjuku.mp4"
TARGET_VIDEO_PATH = "/content/drive/MyDrive/Team7dataset/Team7Shared/140sNightShinjuku_yolov8_result.mp4"

# Initialize video capture
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)

# Video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, fps, (width, height))

# Initialize ByteTrack tracker
byte_track = sv.ByteTrack(
    track_activation_threshold=0.3,
    lost_track_buffer=30,
    frame_rate=fps
)

# Initialize data structures to store past positions for speed estimation
past_positions = defaultdict(lambda: deque(maxlen=5))

# Process the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO object detection
    results = model(frame, conf=0.5)  # Adjust confidence threshold as needed

    # Get detections
    detections = sv.Detections.from_ultralytics(results[0])

    # Filter detections by confidence
    detections = detections[detections.confidence > 0.3]

    # Update tracker with detections
    tracks = byte_track.update_with_detections(detections)

    # Annotate frame manually
    for track in tracks:
        track_id = track[0]
        bbox = track[1]  # Get bounding box coordinates

        if bbox is not None:
            # Draw the bounding box
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            # Calculate speed estimation
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2

            if track_id in past_positions:
                past_positions[track_id].append((center_x, center_y))
                if len(past_positions[track_id]) > 1:
                    # Calculate the displacement between the first and last positions
                    x_start, y_start = past_positions[track_id][0]
                    x_end, y_end = past_positions[track_id][-1]
                    distance = np.sqrt((x_end - x_start)**2 + (y_end - y_start)**2)

                    # Categorize the speed
                    if distance > 50:  # Adjust these thresholds based on your data
                        speed_category = "High Speed"
                        color = (0, 0, 255)  # Red for high speed
                    elif 20 < distance <= 50:
                        speed_category = "Mid Speed"
                        color = (0, 255, 255)  # Yellow for mid speed
                    else:
                        speed_category = "Low Speed"
                        color = (0, 255, 0)  # Green for low speed

                    # Draw the speed category
                    cv2.putText(frame, f'{speed_category}', (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            else:
                past_positions[track_id].append((center_x, center_y))

            # Draw the track ID
            cv2.putText(frame, f'ID: {track_id}', (x1, y1 - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    # Write the annotated frame to the output video
    out.write(frame)

    # Display the frame (optional)
    cv2_imshow(frame)  # Use cv2_imshow instead of cv2.imshow
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

# Save the results
print(f"Results saved to {TARGET_VIDEO_PATH}")


ModuleNotFoundError: No module named 'ultralytics'